In [65]:
import minepsilon as minE

import itertools
import random

import pandas as pd
import numpy as np
import cv2
import torch
import torchvision.transforms as transforms

from pytorchyolo.models import load_model
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
from pytorchyolo.utils.utils import non_max_suppression
from pytorchyolo.utils.loss import compute_loss

In [66]:
df_features = pd.read_csv("compiled_features_dataset.csv")

In [51]:
#my_celeba = ["img_celeba_" + str(i) for i in range(51, 102)]
REF_SET = pd.read_csv(os.path.join(os.getcwd(), 'reference_dataset.csv'), index_col=0)
REF_SET.reset_index()
REF_SET = REF_SET[REF_SET["path"].str.contains("amosc")]
celeba_rows = REF_SET["path"].str.contains("celeba")
REF_SET_celeba = REF_SET[celeba_rows].sample(10)                      #<-- Change face sample count for celeba
REF_SET_wider = REF_SET[celeba_rows == False].sample(10)              #<-- Change face sample count for wi
sample_set = pd.concat([REF_SET_celeba, REF_SET_wider])

In [58]:
# Patterned after FGSM tutorial (https://pytorch.org/tutorials/beginner/fgsm_tutorial.html)
# Define what device we are using
print("CUDA Available: ", torch.cuda.is_available())
device, model = load_model('./weights/yolo_face_sthanhng.cfg', "./weights/yolo_face_sthanhng.weights")

CUDA Available:  False


In [69]:
def detach_cpu(image):
    return image.detach().cpu()

# convert 1x3x416x416 to 416x416x3
def reshape_image(image):
    return np.transpose(np.squeeze(image), (1 ,2, 0))

# convert 1x3x416x416 tensor to 416x416x3 numpy image
def tensor_to_image(image):
    return np.transpose(image.detach().cpu().squeeze().numpy(), (1, 2, 0))

def save_tensor_as_image(image, path):
    save_img = cv2.cvtColor(np.moveaxis((image.detach().numpy() * 255).squeeze(), 0, -1).astype('uint8'), cv2.COLOR_RGB2BGR)
    cv2.imwrite(path, save_img)

In [72]:
nms_scores = [0.1, 0.3, 0.5, 0.7, 0.9]
conf_scores = [0.1, 0.3, 0.5, 0.7, 0.9]
df = pd.DataFrame()
row = {}

for nms, conf in itertools.product(nms_scores, conf_scores):
    row["nms"] = nms
    row["conf"] = conf
    minE.nms_thres = nms
    minE.conf_thres = conf
    
    torch.autograd.set_detect_anomaly(True)
    
    for path in sample_set["path"]:
        row['path'] = path
        REF_SUBSET = sample_set[sample_set['path'] == path]
        
        model.eval()
        model.gradient_mode = False
        for yolo_layer in model.yolo_layers:
            yolo_layer.gradient_mode = False

        # read and transform the image from the path
        data = cv2.imread(path)  # read the image
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB) #change to rgb
        data = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((data, np.zeros((1, 5))))[0].unsqueeze(0) # transform the image

        with torch.no_grad():
            # Forward pass the data through the model and call non max suppression
            nms, nms_output = non_max_suppression(model(data), 0.5, 0.5) #conf_thres and iou_thres = 0.5

        face_list = []
        if type(nms_output[0]) is not int:
            face_list = nms_output[0]

        data = data.to(device)
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        model.gradient_mode = True
        for yolo_layer in model.yolo_layers:
            yolo_layer.gradient_mode = True

        output = model(data)

        # loop through each of the faces in the image
        for face_index, face_row in enumerate(face_list):
            row['face_index'] = face_index
            
            if face_index in set(REF_SUBSET['face_index']):
                x, y, w, h = face_row[0], face_row[1], face_row[2], face_row[3]

                factor_x, factor_y, factor_w, factor_h = random.uniform(1, 2), random.uniform(1, 2), random.uniform(1, 2), random.uniform(1, 2)
                normal_x, normal_y, normal_w, normal_h = x / 416, y / 416, w / 416, h / 416

                new_x = normal_x * factor_x if random.choice([True, False]) else normal_x / factor_x
                new_y = normal_y * factor_y if random.choice([True, False]) else normal_y / factor_y
                new_w = normal_w * factor_w if random.choice([True, False]) else normal_w / factor_w
                new_h = normal_h * factor_h if random.choice([True, False]) else normal_h / factor_h

                new_x, new_y, new_w, new_h = max(min(1, new_x), 0), max(min(1, new_y), 0), max(min(1, new_w), 0), max(min(1, new_h), 0)

                target = torch.tensor([[0.0, 0, new_x, new_y, new_w, new_h]])
                target = target.to(device)

                loss, loss_components = compute_loss(output, target, model)

                # cropped image with bounding box
                # getting (x1, y1) upper left, (x2, y2) lower right
                x1 = max(int(np.floor((x - w / 2).detach().cpu().numpy())), 0)
                y1 = max(int(np.floor((y - h / 2).detach().cpu().numpy())), 0)
                x2 = min(int(np.ceil((x + w / 2).detach().cpu().numpy())), 415)
                y2 = min(int(np.ceil((y + h / 2).detach().cpu().numpy())), 415)

                cropped_image = detach_cpu(data)[:, :, y1:y2, x1:x2] #get the first dimension, the channels, and crop it
                cropped_image = tensor_to_image(cropped_image) #reshape the image to (w/h, h/w, channel)

                # Zero all existing gradients
                model.zero_grad()
                data.grad = None

                # Calculate gradients of model in backward pass
                loss.backward(retain_graph=True)

                # Collect datagrad
                data_grad = data.grad.data

                bbox = (x1, y1, x2, y2)

                bbox_mask = np.zeros(data.shape)
                bbox_mask[..., y1:y2, x1:x2] = 1

                yn_min_e_bbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yn_det_fn, bbox_mask, bbox)
                mp_min_e_bbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.mp_det_fn, bbox_mask, bbox)
                yf_min_e_bbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yf_det_fn, bbox_mask, bbox)
                
                #print("yn min bbox:", yn_min_e_bbox, "mp min bbox:", mp_min_e_bbox, "yf min bbox:", yf_min_e_bbox)
                row['e_bbox_yn'], row['e_bbox_mp'], row['e_bbox_yf'] = yn_min_e_bbox, mp_min_e_bbox, yf_min_e_bbox

                df = df.append(row, ignore_index=True)
df.to_csv('e_from_nms_conf.csv', index=False)

yn min bbox: 0.15 mp min bbox: 0.24499999999999997 yf min bbox: 0.145


C:\Users\amosc\AppData\Local\Temp\ipykernel_5032\2217610674.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)


yn min bbox: 0.15499999999999997 mp min bbox: 0 yf min bbox: 0.10999999999999999


C:\Users\amosc\AppData\Local\Temp\ipykernel_5032\2217610674.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)


yn min bbox: 3.0999999999999974 mp min bbox: 0.16499999999999998 yf min bbox: 0.15499999999999997


...

yn min bbox: 0.15999999999999998 mp min bbox: 0.25499999999999995 yf min bbox: 0.10999999999999999


C:\Users\amosc\AppData\Local\Temp\ipykernel_5032\2217610674.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)


yn min bbox: 0.15999999999999998 mp min bbox: 0 yf min bbox: 0.05500000000000001


C:\Users\amosc\AppData\Local\Temp\ipykernel_5032\2217610674.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)


KeyboardInterrupt: 